In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import tqdm

chipdip_url = 'https://www.chipdip.ru'

group_url = 'https://www.chipdip.ru/catalog/capacitors-lead'

page_str = '?page='

headers = requests.utils.default_headers()

headers.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)

In [2]:
# Получить список разделов конденсаторов
capasitors_sections_links = {}

capasitors_url = 'https://www.chipdip.ru/catalog/resistors'

response = requests.get(capasitors_url , headers=headers)

soup = BeautifulSoup(response.content, "html.parser")

job_elements = soup.find_all("div", class_="no-visited clear catalog")

for job_element in job_elements:

    links = job_element.find_all("a", class_="link")

    for link in links:
        capasitors_sections_links[link.text.strip()] = chipdip_url + link['href']

capasitors_sections_links

{'Резисторы маломощные (до 2 Вт)': 'https://www.chipdip.ru/catalog/rezistory',
 'Резисторы мощные (более 2 Вт)': 'https://www.chipdip.ru/catalog/resistors-2w',
 'Чип резисторы (SMD, для поверхностного монтажа)': 'https://www.chipdip.ru/catalog/smd-resistors',
 'Резисторы переменные': 'https://www.chipdip.ru/catalog-show/potentiometers',
 'Резисторы подстроечные': 'https://www.chipdip.ru/catalog-show/trimmer-potentiometers',
 'Резисторные сборки': 'https://www.chipdip.ru/catalog/resistor-networks',
 'Наборы резисторов': 'https://www.chipdip.ru/catalog-show/resistor-sets'}

In [3]:
# Получить список ссылок на отдельные элементы по страницам с элементами
def extract_links(responses):
    element_links = []

    for response in responses:
        soup = BeautifulSoup(response.content, "html.parser")

        job_elements = soup.find_all("td", class_="h_name")

        for job_element in job_elements[:20]:
            links = job_element.find_all("a", class_="link")

            for link in links:
                element_links.append(chipdip_url + link['href'])

    return element_links

In [5]:
# Получить все страницы в разделе
def get_pages(section, group_url, out_path):
    responses = []

    page = 1
    while True:
        url = group_url + page_str + str(page)
        response = requests.get(url , headers=headers)

        if 'Страница не найдена' in response.text:
            break

        responses.append(response)

        links = extract_links([response])

        with open(out_path, 'a') as outfile:
            for l in links:
                outfile.write(section + ',"' + l + '"\n')

        page += 1

        time.sleep(random.randrange(3, 10))

    # return responses

In [18]:
get_pages('Конденсаторы силовые', capasitors_sections_links['Конденсаторы силовые'].replace('-show', ''))

In [7]:
for name, link in capasitors_sections_links.items():
    if 'catalog-show' in link:
        print(name)

Конденсаторы силовые
Наборы конденсаторов
Конденсаторы для светотехники


In [6]:
for section, link in capasitors_sections_links.items():
    get_pages(section, link.replace('-show', ''), 'resistor_links.csv')

In [11]:
import pandas as pd
path = 'resistor_links.csv'
df = pd.read_csv(path)
display(df)

,section_name,link
0,Резисторы маломощные (до 2 Вт),https://www.chipdip.ru/product0/11677
1,Резисторы маломощные (до 2 Вт),https://www.chipdip.ru/product0/28651
2,Резисторы маломощные (до 2 Вт),https://www.chipdip.ru/product0/6699
3,Резисторы маломощные (до 2 Вт),https://www.chipdip.ru/product0/35686
4,Резисторы маломощные (до 2 Вт),https://www.chipdip.ru/product0/20427
...,...,...
6242,Наборы резисторов,https://www.chipdip.ru/product0/8006877722
6243,Наборы резисторов,https://www.chipdip.ru/product0/8006881104
6244,Наборы резисторов,https://www.chipdip.ru/product0/8005160449
6245,Наборы резисторов,https://www.chipdip.ru/product0/8006909964


In [12]:
df2 = df.groupby(['section_name'])['section_name'].count()
display(df2)

section_name
Наборы резисторов                                   247
Резисторные сборки                                 1000
Резисторы маломощные (до 2 Вт)                     1000
Резисторы мощные (более 2 Вт)                      1000
Резисторы переменные                               1000
Резисторы подстроечные                             1000
Чип резисторы (SMD, для поверхностного монтажа)    1000
Name: section_name, dtype: int64

In [43]:
# example
links = extract_links(get_pages(capasitors_sections_links['Конденсаторы ниобиевые']))
links

['https://www.chipdip.ru/product/k53-19-3.3mkf-h-20v-20',
 'https://www.chipdip.ru/product0/8025663496',
 'https://www.chipdip.ru/product0/8021838883',
 'https://www.chipdip.ru/product0/8021838884',
 'https://www.chipdip.ru/product0/8021838881',
 'https://www.chipdip.ru/product0/8025666624',
 'https://www.chipdip.ru/product0/8021838879',
 'https://www.chipdip.ru/product0/8021838882',
 'https://www.chipdip.ru/product0/8025663494',
 'https://www.chipdip.ru/product0/8021838880',
 'https://www.chipdip.ru/product0/8005649538',
 'https://www.chipdip.ru/product/noja475m010rwj',
 'https://www.chipdip.ru/product0/8009246798',
 'https://www.chipdip.ru/product0/8015828446',
 'https://www.chipdip.ru/product0/8006475558',
 'https://www.chipdip.ru/product0/8009269676',
 'https://www.chipdip.ru/product0/8006280876',
 'https://www.chipdip.ru/product/nosb226m006r0600',
 'https://www.chipdip.ru/product0/8009269678']

In [ ]:
d = {}

for group_name, group_url in tqdm(capasitors_sections_links.items()):
    links = extract_links(get_pages(group_url))

    d[group_name] = links


In [25]:
# достать из страницы текст с описанием и характеристиками
url = 'https://www.chipdip.ru/product/tzc3p200a110'

response = requests.get(url , headers=headers)

soup = BeautifulSoup(response.content, "html.parser")

header_class = 'main-header'
describtion_class = 'showhide item_desc'
characteristics_class = 'product__params-w'

header = ''
describtion = ''
characteristics = ''

job_elements = soup.find_all("div", class_=header_class)
for job_element in job_elements:
    header = job_element.text.strip()

job_elements = soup.find_all("div", class_=describtion_class)
for job_element in job_elements:
    describtion = job_element.text.strip()


job_elements = soup.find_all("div", class_=characteristics_class)
for job_element in job_elements:
    strs = job_element.find_all("tr")

    for s in strs:
        characteristics += s.text.strip().replace('\n', ' ') + '\n'

print(header)
print(describtion)
print(characteristics)

TZC3P200A110**, 5-20пФ, SMD конденсатор подстроечный
Компактные подстроечные конденсаторы для поверхностного монтажа Разработаны для автоматического монтажа Устойчивы к высоким температурам Рабочее напряжение 100В Максимальное напряжение 220В Температурный диапазон от -25 до + 85 °C
Тип tzc3  ​
Рабочее напряжение,В 100
Емкость мин.,пкФ 5  ​
Емкость макс.,пФ 20  ​
Температурный коэффициент емкости(ТКЕ) n1200
Рабочая температура,С -25…85
Добротность Qмин. 300
Размер корпуса ,мм 3.2x4.5
Вес, г 0.05



In [ ]:
print(describtion)
print(characteristics)